In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from src import parser
import numpy as np

In [6]:
a = np.array([[1,2], [3,4]])

In [55]:
m = np.array([[0,-1], [1,0]])
i = np.array([[1,0],[0,1]])

In [92]:
b = parser.transform_block_array(a,m)

In [93]:
a

array([[1, 2],
       [3, 4]])

In [94]:
b

array([[2., 4.],
       [1., 3.]])

In [95]:
parser.transform_block_array(a,i)

array([[1., 2.],
       [3., 4.]])

In [96]:
np.rot90(a)

array([[2, 4],
       [1, 3]])

In [97]:
parser.transform_block_array(b, m)

array([[4., 3.],
       [2., 1.]])

In [98]:
np.flip(a, 0)

array([[3, 4],
       [1, 2]])

In [99]:
f = np.array([[-1, 0], [0,1]])

In [100]:
parser.transform_block_array(a, f)

array([[3., 4.],
       [1., 2.]])